# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
import numpy as np
import os
from os import listdir
#from scipy.misc import imread, imresize
import datetime
import os
import cv2 as cv
import random as rn
import tensorflow as tf
from tensorflow import keras
#from keras import backend as K
from os import listdir

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)

rn.seed(30)

tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 50 #experiment with the batch size

In [ ]:
print(len(train_doc))
print(len(val_doc))

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

## Custom code for demo on image processing
Below i have created a custom code just to demo on reading a random image , then croping it , Resizing and normlising it.

In [ ]:
import matplotlib.pyplot as plt
import random
train_fol = '/home/datasets/Project_data/train'
samp = np.random.permutation(train_doc)
samp_folder = random.choice(samp).split(';')[0]
samp_imgs = os.listdir(train_fol+'/'+ samp_folder)
samp_image = cv.imread(train_fol+'/'+ samp_folder+'/'+samp_imgs[0])[...,::-1]
h, w ,c  = samp_image.shape
plt.figure(figsize=(12, 12))
ax = plt.subplot(1, 4, 1)
plt.title('Original Image')
plt.imshow(samp_image)
crop_samp = samp_image[int(h*.20) : h-int(h*.05), int(w*.05) : w-int(w*.05)]
ax = plt.subplot(1, 4, 2)
plt.title('Cropped Image')
plt.imshow(crop_samp)
re_sam_image = cv.resize(crop_samp, (150,150), interpolation = cv.INTER_AREA)
ax = plt.subplot(1, 4, 3)
plt.title('Resized Image')
plt.imshow(re_sam_image)
norm_samp = np.zeros((150,150))
final_sam = cv.normalize(re_sam_image,  norm_samp, 0, 255, cv.NORM_MINMAX)
ax = plt.subplot(1, 4, 4)
plt.title('Normalised Image')
plt.imshow(final_sam)
plt.show()

In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [1,3,5,7,9,11,12,13,14,15,16,17,19,20,21,22,24,26,28,29]
    y=150
    z=150
    a = len(img_idx) 
    while True:
        t = np.random.permutation(folder_list)
        fol = list(t)
        num_batches = len(t)//batch_size
        for batch in range(num_batches): 
            batch_data = np.zeros((batch_size,a,y,z,3))  
            batch_labels = np.zeros((batch_size,5))
            for folder in range(batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                fol.remove(t[folder + (batch*batch_size)])
                for idx,item in enumerate(img_idx):
                    image = cv.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])[...,::-1]
                    height, width ,channels  = image.shape
                    ### Crop image as well crop to or side etc
                    #Left_right_Swipe
                    if int(t[folder + (batch*batch_size)].strip().split(';')[2]) in (0,1): 
                        crop = image[int(height*.20) : height-int(height*.05), int(width*.05) : width-int(width*.05)] 
                    # Rest of the class
                    else:
                        crop = image[int(height*.10) : height-int(height*.05) ,int(width*.20) : width-int(width*.20)]
                    #Resize image to 150*150
                    re_image = cv.resize(crop, (y,z), interpolation = cv.INTER_AREA)
                    # normalise image
                    norm = np.zeros((y,z))
                    final = cv.normalize(re_image,  norm, 0, 255, cv.NORM_MINMAX)
                    batch_data[folder,idx,:,:,0] = final[:,:,0]
                    batch_data[folder,idx,:,:,1] =  final[:,:,1]
                    batch_data[folder,idx,:,:,2] = final[:,:,2]
        
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels 
                    #code for the remaining data points which are left after full batches
        pen_fol = len(fol)
        batch_data = np.zeros((pen_fol,a,y,z,3))  
        batch_labels = np.zeros((pen_fol,5))
        for folder in range(pen_fol):
            sel_fol = fol[folder].split(';')[0]
            imgs = os.listdir(source_path+'/'+ sel_fol)
            for idx,item in enumerate(img_idx):
                image = cv.imread(source_path+'/'+ sel_fol+'/'+imgs[item])[...,::-1]
                height, width ,channels  = image.shape
                ### Crop image as well crop to or side etc
                #Left_right_Swipe
                if int(fol[folder].split(';')[2]) in (0,1): 
                    crop = image[int(height*.20) : height-int(height*.05), int(width*.05) : width-int(width*.05)]
                # Rest of the class
                else:
                    crop = image[int(height*.10) : height-int(height*.05) ,int(width*.20) : width-int(width*.20)]
                #Resize image to 140*140
                re_image = cv.resize(crop, (y,z), interpolation = cv.INTER_AREA)
                norm = np.zeros((y,z))
                final = cv.normalize(re_image,  norm, 0, 255, cv.NORM_MINMAX)
                # normalise image
                norm = np.zeros((y,z))
                final = cv.normalize(re_image,  norm, 0, 255, cv.NORM_MINMAX)
                batch_data[folder,idx,:,:,0] = final[:,:,0] #normalise and feed in the image
                batch_data[folder,idx,:,:,1] = final[:,:,1]#normalise and feed in the image
                batch_data[folder,idx,:,:,2] = final[:,:,2]#normalise and feed in the image
            batch_labels[folder, int(fol[folder].split(';')[2])] = 1
        yield batch_data, batch_labels       


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 25 # choose the number of epochs  atleast 50 
print ('# epochs =', num_epochs)

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import Conv3D, MaxPooling3D,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2
#write your model here

In [ ]:
sample_shape = (20,150, 150, 3)
model = Sequential()
model.add(Conv3D(32,kernel_size=(3,3,3),activation='relu',input_shape=sample_shape))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu',kernel_regularizer=l2(0.01),padding="same"))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(5,activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False,save_freq="epoch", mode='auto')
#num_epochs
LR = 0.001 # write the REducelronplateau code here
callbacks_list = [checkpoint]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
print(steps_per_epoch,validation_steps)

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()